# Interfaces, Implementations, and Testing

## Design of a program

From the Practice of Programming:

>The essence of design is to balance competing goals and constraints. Although there may be many tradeoffs when one is writing a small self-contained system, the ramifications of particular choices remain within the system and affect only the indi- vidual programmer. But when code is to be used by others, decisions have wider repercussions.


What are the issues we need to be cognizant of?

- **Interfaces**: your program is being designed to be used by someone: either an end user, another programmer (you are writing a library), or even yourself (we are talking about some layer in your program). This **interface** is a contract between you and the user: with preconditions, postconditions, etc
- There is **information** hiding between layers (a higher up layer can be more abstract, more lossy about all the information). Encapsulation,  abstraction,  modularization, are some of the techniques used here.
- There are **resource management** issues: who allocates storage for data structures.. (generally we want resource allocation/deallocation to happen in the same layer)
- How to **deal with errors**: do we return special values, throw exceptions? who handles them? (generally we want to catch even the lowest level error and give the client the chance to handle it, possibly lossily)

### Interface principles

Interfaces should:

- hide implementation details
- have a small set of operations exposed, the smallest possible, and these should be orthogonal. Be stingy with the user.
- but be transparent with the user in what goes on behind the scenes (calls the NSA)
- be consistent internally: library functions should have similar signature, classes similar methods, and externally: programs should have the same cli flags, same ops.

** Testing should deal with ALL of the issues above, and each layer ought to be tested separately **. 

This gives rise to :

### Different kinds of tests

- **acceptance tests** verify that a program meets a customer's expectations. In a sense these are a test of the *interface* to the customer: does the program do everything you promised the customer it would do? You might use test harnesees for cli programs and selenium for this. The test of a library interface could also be thought of as an acceptance test

- **unit tests** are tests which test a unit of the program, for use by another unit. These could test the interface for a client, but must also be testing internal functions which you want to use.

Exploratory testing, regression testing, and integration testing are done in both of these categories, with the latter trying to combine layers and subsystems, not necessarily at the level of an entire application. 

One can also performance test, random and exploratorily test, and stress test a system (to create adversarial situations).

## Testing of a program

Test as you write your program.

This is so important that I repeat it.

**Test as you go**.

You will cry otherwise. I have in the past.

From The Practice of Programming:


>The  effort  of  testing as  you  go  is  minimal  and  pays off  handsomely.  Thinking about testing as you  write a program will  lead to better code, because that's when you know  best  what the code should do.  If  instead  you  wait  until  something breaks, you will  probably  have forgotten how  the code works.  Working under  pressure, you  will need  to figure it  out again, which  takes time, and  the fixes  will  be  less  thorough  and more fragile because your refreshed understanding is  likely to be incomplete. 

### Assertions and the process of testing

The workhorse of testing is the `assert` statement. The same statement can also be used to assert preconditions and postconditions, and thus to test them if appropriate. In C, `assert` is a macro which can be conditionally compiled away.

In [1]:
def myaverage(l:list)->float:
    sumit = 0.0
    for f in l:
        sumit = sumit + f
    average = sumit/len(l)
    return average

In [2]:
assert myaverage([1,2])==1.5

In [3]:
assert myaverage([1,2])==3

AssertionError: 

### Principles of testing

#### Test Simple Parts First

In [4]:
def test_average():
    assert myaverage([1,2])==1.5, "1 and 2 must average to 1.5"

In [5]:
test_average()

In [6]:
def myaverage(l:list)->float:
    average = 0.0
    for f in l:
        average = average + 2*f
    average = average/len(l)
    return average

In [7]:
test_average()

AssertionError: 1 and 2 must average to 1.5

In [8]:
def myaverage(l:list)->float:
    n = len(l)
    thesum = sum(l)
    average = thesum/n
    return average

In [9]:
test_average()

#### Test code at its boundaries

The idea is that most errors happen at data boundaries such as empty input, single input item, exactly full array, wierd values, etc. If a piece of code works at the boundaries, its likely to work elsewhere...


In [10]:
myaverage([])

ZeroDivisionError: division by zero

#### Program defensively 

The user is supposed to give us an array, non-zero. We could specify that as a precondition, but we might as well be defensive and test for that.

Practice:
>"Program  defensively.  A  useful  technique is  to  add  code  to  handle  "can't  happen" cases,  situations  where  it  is  not  logically  possible  for  something  to  happen  but (because of  some failure elsewhere) it might anyway.  Adding a test for zero or nega- tive array lengths to avg  was one example.  As another example, a program  process- ing  grades might  expect  that  there  would  be  no  negative  or huge  values  but  should check anyway: 

 From https://docs.python.org/3/library/exceptions.html:
 
 >exception **ValueError**
 
>Raised when a built-in operation or function receives an argument that has the right type but an inappropriate value, and the situation is not described by a more precise exception such as IndexError.

In [11]:
def myaverage(l:list)->float:
    """
    Calculate the average of list l
    """
    n = len(l)
    if n==0:
        raise ValueError("cant calculate mean of length 0 list")
    thesum = sum(l)
    average = thesum/n
    return average

In [12]:
test_average()

In [41]:
try:
    myaverage([])
except Exception as e:
    print("hi",type(e), e.args)

hi <class 'ValueError'> ('cant calculate mean of length 0 list',)


In [43]:
def test_average_empty():
    try:
        myaverage([])
    except Exception as e:
        assert (type(e) == ValueError and e.args[0]=='cant calculate mean of length 0 list')

In [44]:
test_average_empty()

#### Automate using a test harness

Doctests are one way to do this. 


In [47]:
%%file mymath.py


def myaverage(l:list)->float:
    """
    Calculate the average of list l
    
    Examples:
    
    >>> myaverage([1,2])
    1.5
    >>> myaverage([])
    Traceback (most recent call last):
        ...
    ValueError: cant calculate mean of length 0 list
    """
    n = len(l)
    if n==0:
        raise ValueError("cant calculate mean of length 0 list")
    thesum = sum(l)
    average = thesum/n
    return average

Overwriting mymath.py


In [48]:
!python3 -m doctest mymath.py --verbose

Trying:
    myaverage([1,2])
Expecting:
    1.5
ok
Trying:
    myaverage([])
Expecting:
    Traceback (most recent call last):
        ...
    ValueError: cant calculate mean of length 0 list
ok
1 items had no tests:
    mymath
1 items passed all tests:
   2 tests in mymath.myaverage
2 tests in 2 items.
2 passed and 0 failed.
Test passed.


#### The unittest framework

But too many doctests clutter the documentation of a class.

One should only have those examples as doctests which describe the various ways a class or function can be used. Edge cases which must work, etc, ought to be represented in a separate test file

In [50]:
%%file test_mymath.py


import unittest

from mymath import myaverage

class MyTest(unittest.TestCase):
    
    def test_mymath(self):
        self.assertEqual(myaverage([2,3]), 2.5)

if __name__ == '__main__':
    unittest.main()

Writing test_mymath.py


In [51]:
!python3 test_mymath.py

.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


#### When you get an error

It could be that:

- you messed up an implementation (we saw this above with the mult by two)
- if the error was not found in an existing test, create a new test that represents the problem BEFORE you do anything else. The test shold capture the essence of the problem: this process itself is useful in uncovering bugs. Then this error may even suggest more tests. You fix, perhaps by writing defensive code.


In [52]:
myaverage(['a',1])

TypeError: unsupported operand type(s) for +: 'int' and 'str'

#### Test Incrementally

In this way you test incrementally, adding tests all the time.

In [53]:
import unittest

class MyTest(unittest.TestCase):
    
    def test_mymath(self):
        self.assertEqual(myaverage([2,3]), 2.5)
        
    def test_char(self):
        with self.assertRaises(TypeError):
            myaverage(['a',3])
            
    def test_zerol(self):
        with self.assertRaises(ValueError):
            myaverage([])


suite = unittest.TestLoader().loadTestsFromModule(MyTest())
unittest.TextTestRunner().run(suite)

...
----------------------------------------------------------------------
Ran 3 tests in 0.001s

OK


<unittest.runner.TextTestResult run=3 errors=0 failures=0>

### Test pre-conditions and post-conditions

While we could take the position that bad pre-conditions lead to undefined behavior, we are good citizens if we test preconditions for the user...and this helps during algorithm development as well. Here we test 2 pre-conditions: no zero length, and numerical array. The latter would be too expensive at the beginning, so we do it by wrapping `sum` in a `try..except` block. We could test postcondition by asserting that the average is a number.

In [54]:
def myaverage(l:list)->float:
    """
    Calculate the average of list l
    
    Examples:
    
    >>> myaverage([1,2])
    1.5
    
    """
    n = len(l)
    if n==0:
        raise ValueError("cant calculate mean of length 0 list")
    try:
        thesum = sum(l)
    except:
        raise TypeError("Cannot sum things of different types")
    average = thesum/n
    return average

In [55]:
myaverage(['a',1])

TypeError: Cannot sum things of different types

In [93]:
%%file mymath.py
def myaverage(l:list)->float:
    """
    Calculate the average of list l
    
    Examples:
    
    >>> myaverage([1,2])
    1.5
    
    """
    n = len(l)
    if n==0:
        raise ValueError("cant calculate mean of length 0 list")
    try:
        thesum = sum(l)
    except:
        raise TypeError("Cannot sum things of different types")
    average = thesum/n
    return average

Overwriting mymath.py


In [94]:
from fractions import Fraction
myaverage([Fraction(1,3), Fraction(2,3)])

Fraction(1, 2)

#### Working the `unittest` module

In [95]:
%%file test_mymath.py
from mymath import myaverage
import unittest
import numbers
from fractions import Fraction

class MyTest(unittest.TestCase):
    
    def test_mymath(self):
        self.assertEqual(myaverage([2,3]), 2.5)
        
    def test_mymath_result(self):
        self.assertTrue(isinstance(myaverage([Fraction(1,3), Fraction(2,3)]), numbers.Real))

    def test_char(self):
        with self.assertRaises(TypeError):
            myaverage(['a',3])
            
    def test_zerol(self):
        with self.assertRaises(ValueError):
            myaverage([])


if __name__ == '__main__':
    unittest.main()

Overwriting test_mymath.py


In [96]:
!python3 -m unittest

....
----------------------------------------------------------------------
Ran 4 tests in 0.001s

OK


In [97]:
!python3 -m unittest test_mymath

....
----------------------------------------------------------------------
Ran 4 tests in 0.000s

OK


### A test harness: The `py.test` program

As a group you should choose a test harness. I like `py.test`: it will run some `nosetes`s, `unittest`s, as well as its own set. Here we talk about `py.test`, but the principles are the same for any such framework.

Install thus. Make sure you are in the `py35` virtual environment.

`pip install pytest`

`pip install pytest-cov`

In [98]:
!py.test

============================= test session starts ==============================
platform darwin -- Python 3.5.1, pytest-2.8.1, py-1.4.30, pluggy-0.3.1
rootdir: /Users/rahul/Projects/private/cs207/lecswithlabs/week5, inifile: 
plugins: cov-2.2.1
collected 4 items 

test_mymath.py ....

=========================== 4 passed in 0.03 seconds ===========================


In [99]:
!py.test --doctest-modules 

============================= test session starts ==============================
platform darwin -- Python 3.5.1, pytest-2.8.1, py-1.4.30, pluggy-0.3.1
rootdir: /Users/rahul/Projects/private/cs207/lecswithlabs/week5, inifile: 
plugins: cov-2.2.1
collected 5 items 

mymath.py .
test_mymath.py ....

=========================== 5 passed in 0.03 seconds ===========================


#### Testing and coverage

In some sense, it would be nice to somehow check that every line in a program has been covered by a test, so that you know that line has not contributed to making something wrong. But this is hard to do: it would be hard to use normal input data to force a program to go through particular statements. So we settle for testing the important lines. The `pytest-cov` module makes sure that this works.

Coverage does not mean that every edge case has been tried, but rather, every critical statement has been.

We add a median function to our file:

In [107]:
def mymedian(l:list)->float:
    """
    Calculate the average of list l
    
    Examples:
    
    >>> mymedian([1,2,3])
    2
    
    >>> mymedian([1,2,3,4])
    2.5
    """
    lsorted = sorted(l)
    mididx = len(lsorted)//2
    if len(lsorted) % 2 == 0: #even
        return (lsorted[mididx-1] + lsorted[mididx])/2
    else:
        return lsorted[mididx]

In [108]:
%%file mymath.py

def myaverage(l:list)->float:
    """
    Calculate the average of list l
    
    Examples:
    
    >>> myaverage([1,2])
    1.5
    
    """
    n = len(l)
    if n==0:
        raise ValueError("cant calculate mean of length 0 list")
    try:
        thesum = sum(l)
    except:
        raise TypeError("Cannot sum things of different types")
    average = thesum/n
    return average

def mymedian(l:list)->float:
    """
    Calculate the average of list l
    
    Examples:
    
    >>> mymedian([1,2,3])
    2
    
    >>> mymedian([1,2,3,4])
    2.5
    """
    lsorted = sorted(l)
    mididx = len(lsorted)//2
    if len(lsorted) % 2 == 0: #even
        return (lsorted[mididx-1] + lsorted[mididx])/2
    else:
        return lsorted[mididx]

Overwriting mymath.py


#### Using `py.test` with coverage

In [114]:
!cat test_mymath.py

from mymath import myaverage, mymedian
import unittest
import numbers
from fractions import Fraction

class MyTest(unittest.TestCase):
    
    def test_mymath(self):
        self.assertEqual(myaverage([2,3]), 2.5)
        
    def test_mymath_result(self):
        self.assertTrue(isinstance(myaverage([Fraction(1,3), Fraction(2,3)]), numbers.Real))

    def test_char(self):
        with self.assertRaises(TypeError):
            myaverage(['a',3])
            
    def test_zerol(self):
        with self.assertRaises(ValueError):
            myaverage([])


if __name__ == '__main__':
    unittest.main()

In [110]:
!py.test --doctest-modules --cov --verbose

============================= test session starts ==============================
platform darwin -- Python 3.5.1, pytest-2.8.1, py-1.4.30, pluggy-0.3.1 -- //anaconda/envs/py35/bin/python
cachedir: .cache
rootdir: /Users/rahul/Projects/private/cs207/lecswithlabs/week5, inifile: 
plugins: cov-2.2.1
collected 6 items 

mymath.py::mymath.myaverage PASSED
mymath.py::mymath.mymedian PASSED
test_mymath.py::MyTest::test_char PASSED
test_mymath.py::MyTest::test_mymath PASSED
test_mymath.py::MyTest::test_mymath_result PASSED
test_mymath.py::MyTest::test_zerol PASSED
--------------- coverage: platform darwin, python 3.5.1-final-0 ----------------
Name             Stmts   Miss  Cover
------------------------------------
mymath.py           16      0   100%
test_mymath.py      17      1    94%
------------------------------------
TOTAL               33      1    97%

=========================== 6 passed in 0.09 seconds ===========================


And you can ask for a coverage report with missing lines

In [111]:
!py.test --doctest-modules --cov --cov-report term-missing

============================= test session starts ==============================
platform darwin -- Python 3.5.1, pytest-2.8.1, py-1.4.30, pluggy-0.3.1
rootdir: /Users/rahul/Projects/private/cs207/lecswithlabs/week5, inifile: 
plugins: cov-2.2.1
collected 6 items 

mymath.py ..
test_mymath.py ....
--------------- coverage: platform darwin, python 3.5.1-final-0 ----------------
Name             Stmts   Miss  Cover   Missing
----------------------------------------------
mymath.py           16      0   100%   
test_mymath.py      17      1    94%   24
----------------------------------------------
TOTAL               33      1    97%   

=========================== 6 passed in 0.06 seconds ===========================


#### `mymedian` has its problems

In [112]:
mymedian(['a',1])

TypeError: unorderable types: int() < str()

In [113]:
mymedian([])

IndexError: list index out of range

Lets fix by adressing preconditions in a similar way.

In [115]:
%%file mymath.py

def myaverage(l:list)->float:
    """
    Calculate the average of list l
    
    Examples:
    
    >>> myaverage([1,2])
    1.5
    
    """
    n = len(l)
    if n==0:
        raise ValueError("cant calculate mean of length 0 list")
    try:
        thesum = sum(l)
    except:
        raise TypeError("Cannot sum things of different types")
    average = thesum/n
    return average

def mymedian(l:list)->float:
    """
    Calculate the average of list l
    
    Examples:
    
    >>> mymedian([1,2,3])
    2
    
    >>> mymedian([1,2,3,4])
    2.5
    """
    try:
        lsorted = sorted(l)
    except:
        raise TypeError("Unable to sort array")
    n = len(lsorted)
    if n==0:
        raise ValueError("cant calculate median of length 0 list")
    mididx = len(lsorted)//2
    if len(lsorted) % 2 == 0: #even
        return (lsorted[mididx-1] + lsorted[mididx])/2
    else:
        return lsorted[mididx]

Overwriting mymath.py


From Practice:

>The  effort  of  testing as  you  go  is  minimal  and  pays off  handsomely.  Thinking about testing as you  write a program will  lead to better code, because that's when you know  best  what the code should do.  If  instead  you  wait  until  something breaks, you will  probably  have forgotten how  the code works.  Working under  pressure, you  will need  to figure it  out again, which  takes time, and  the fixes  will  be  less  thorough  and more fragile because your refreshed understanding is  likely to be incomplete. 

### Back to py.test

- pytest runs doctests and unittests.
- any function prefixed with `test_` is a test.
- will tey nosetests

(you dont have to use py.test, and can use nosetests if your team prefers it)

You can continue to use `unittest`. `py.test` has the advantage that simple asserts are transformed the way stuff like `assertEqual` works. Not only that, `py.test` gives you some useful message as to why the asertion failed.

In [120]:
%%file test_mymath2.py

from pytest import raises
from mymath import myaverage, mymedian

def test_mymath_mean():
    assert myaverage([9,3]) == 6

def test_char():
    with raises(TypeError):
        myaverage(['a',3])

def test_mymath():
    assert mymedian([9,3, 6]) == 5
    
def test_zero_median():
    with raises(ValueError):
        mymedian([])
        
def test_char_median():
    with raises(TypeError):
        mymedian(['a', 3])

Overwriting test_mymath2.py


#### Know the answer or get it simply otherwise

Be very careful that your test suite is correct. The above is not, and whether the test suite is wrong, or our functions are not doing the right thing, `py.test` gives a useful message. If you need to calculate an answer, use a simpler method.

In [121]:
!py.test --doctest-modules --cov --cov-report term-missing

============================= test session starts ==============================
platform darwin -- Python 3.5.1, pytest-2.8.1, py-1.4.30, pluggy-0.3.1
rootdir: /Users/rahul/Projects/private/cs207/lecswithlabs/week5, inifile: 
plugins: cov-2.2.1
collected 11 items 

mymath.py ..
test_mymath.py ....
test_mymath2.py ..F..

=================================== FAILURES ===================================
_________________________________ test_mymath __________________________________

    def test_mymath():
>       assert mymedian([9,3, 6]) == 5
E       assert 6 == 5
E        +  where 6 = mymedian([9, 3, 6])

test_mymath2.py:13: AssertionError
--------------- coverage: platform darwin, python 3.5.1-final-0 ----------------
Name              Stmts   Miss  Cover   Missing
-----------------------------------------------
mymath.py            22      0   100%   
test_mymath.py       17      1    94%   24
test_mymath2.py      15      0   100%   
-----------------------------------------------
TO

We fix it

In [123]:
%%file test_mymath2.py

from pytest import raises
from mymath import myaverage, mymedian

def test_mymath_mean():
    assert myaverage([9,3]) == 6

def test_char():
    with raises(TypeError):
        myaverage(['a',3])

def test_mymath():
    assert mymedian([9,3, 6]) == 6
    
def test_zero_median():
    with raises(ValueError):
        mymedian([])
        
def test_char_median():
    with raises(TypeError):
        mymedian(['a', 3])

Overwriting test_mymath2.py


In [124]:
!py.test --doctest-modules --cov --cov-report term-missing

============================= test session starts ==============================
platform darwin -- Python 3.5.1, pytest-2.8.1, py-1.4.30, pluggy-0.3.1
rootdir: /Users/rahul/Projects/private/cs207/lecswithlabs/week5, inifile: 
plugins: cov-2.2.1
collected 11 items 

mymath.py ..
test_mymath.py ....
test_mymath2.py .....
--------------- coverage: platform darwin, python 3.5.1-final-0 ----------------
Name              Stmts   Miss  Cover   Missing
-----------------------------------------------
mymath.py            22      0   100%   
test_mymath.py       17      1    94%   24
test_mymath2.py      15      0   100%   
-----------------------------------------------
TOTAL                54      1    98%   

========================== 11 passed in 0.05 seconds ===========================


### TDD

From Younker's AgilePython Development:

>TDD uses very small development cycles. Tests aren’t written for entire functions. They are written incrementally as the functions are composed. If the chunks get too large, a test- driven developer can always back down to a smaller chunk.
The cycles have a distinct four-part rhythm. A test is written, and then it is executed to verify that it fails. A test that succeeds at this point tells you nothing about your new code. (Every day I encounter one that works when I don’t expect it to.) After the test fails, the associ- ated code is written, and then the test is run again. This time it should pass. If it passes, then the process begins anew.

One advantage of this is that focusses you on the interface of your function/class, etc. The downside is that it precludes exploration and exploration tests, where you might try different interfaces. Another danger is write you might write tons of small units.

Should you do it? TDD is certainly useful at times, but make sure you do it when you have a concrete idea where you are going...

### Fixtures: setup common scaffolding for your tests

Many times you need a common setup and teardown for multiple tests. You might need to populate some data, for example. This is done via fixtures.



In [125]:
%%file test_mymath3.py


from pytest import fixture
from mymath import myaverage, mymedian


@fixture
def input_data():
    return dict(b=[4,5,6], a=['a', 1,2])

def test_with_fixture(input_data):
    assert myaverage(input_data['b']) == 5
    assert mymedian(input_data['b']) == 5

Writing test_mymath3.py


In [126]:
!py.test --doctest-modules --cov --cov-report term-missing

============================= test session starts ==============================
platform darwin -- Python 3.5.1, pytest-2.8.1, py-1.4.30, pluggy-0.3.1
rootdir: /Users/rahul/Projects/private/cs207/lecswithlabs/week5, inifile: 
plugins: cov-2.2.1
collected 19 items 

amath.py ..
mymath.py ..
test_amath.py .....
test_mymath.py ....
test_mymath2.py .....
test_mymath3.py .
--------------- coverage: platform darwin, python 3.5.1-final-0 ----------------
Name              Stmts   Miss  Cover   Missing
-----------------------------------------------
amath.py             22      1    95%   14
mymath.py            22      0   100%   
test_amath.py        15      0   100%   
test_mymath.py       17      1    94%   24
test_mymath2.py      15      0   100%   
test_mymath3.py       7      0   100%   
-----------------------------------------------
TOTAL                98      2    98%   

========================== 19 passed in 0.09 seconds ===========================


In [127]:
%%file test_mymath4.py


from pytest import fixture
from mymath import mymedian, myaverage


@fixture(scope="module")
def input_data():
    return dict(b=range(1000))

def test_first(input_data):
    assert mymedian(input_data['b'])  == 499.5

def test_second(input_data):
    assert myaverage(input_data['b']) == 499.5

Writing test_mymath4.py


In [128]:
!py.test --doctest-modules test_mymath4.py

============================= test session starts ==============================
platform darwin -- Python 3.5.1, pytest-2.8.1, py-1.4.30, pluggy-0.3.1
rootdir: /Users/rahul/Projects/private/cs207/lecswithlabs/week5, inifile: 
plugins: cov-2.2.1
collected 2 items 

test_mymath4.py ..

=========================== 2 passed in 0.02 seconds ===========================


In [129]:
%%file test_mymath5.py
from pytest import fixture
import io
#from docs
@fixture
def file_data(request): # The fixture MUST have a 'request' argument
    text = open("test_mymath5.py")

    @request.addfinalizer
    def teardown():
        text.close()
    return text

def test_data_type(file_data):
    assert isinstance(file_data, io.TextIOWrapper)

Writing test_mymath5.py


In [131]:
!py.test --doctest-modules test_mymath5.py

============================= test session starts ==============================
platform darwin -- Python 3.5.1, pytest-2.8.1, py-1.4.30, pluggy-0.3.1
rootdir: /Users/rahul/Projects/private/cs207/lecswithlabs/week5, inifile: 
plugins: cov-2.2.1
collected 1 items 

test_mymath5.py .

=========================== 1 passed in 0.02 seconds ===========================


You can use fixtures in `unittest` as well..but they look a little bit different.

In [132]:
%%file test_mymath6.py

from mymath import myaverage, mymedian

import unittest

class MyTest(unittest.TestCase):
    
    def setUp(self):
        self.b = range(1000)
        
    def tearDown(self):
        del self.b
        
    def test_mymath(self):
        self.assertEqual(myaverage(self.b), 499.5)
        
    def test_char(self):
        with self.assertRaises(TypeError):
            myaverage(['a',3])

Writing test_mymath6.py


In [133]:
!py.test --doctest-modules --cov --cov-report term-missing

============================= test session starts ==============================
platform darwin -- Python 3.5.1, pytest-2.8.1, py-1.4.30, pluggy-0.3.1
rootdir: /Users/rahul/Projects/private/cs207/lecswithlabs/week5, inifile: 
plugins: cov-2.2.1
collected 24 items 

amath.py ..
mymath.py ..
test_amath.py .....
test_mymath.py ....
test_mymath2.py .....
test_mymath3.py .
test_mymath4.py ..
test_mymath5.py .
test_mymath6.py ..
--------------- coverage: platform darwin, python 3.5.1-final-0 ----------------
Name              Stmts   Miss  Cover   Missing
-----------------------------------------------
amath.py             22      1    95%   14
mymath.py            22      0   100%   
test_amath.py        15      0   100%   
test_mymath.py       17      1    94%   24
test_mymath2.py      15      0   100%   
test_mymath3.py       7      0   100%   
test_mymath4.py       8      0   100%   
test_mymath5.py       9      0   100%   
test_mymath6.py      12      0   100%   
----------------------

### Fakes and Mocks: make your tests self-contained

But in general you shouldnt even be dealing with a file or db. u should **mock** it out. Your tests should nor rely on a database being there, or a network connection being possible. Otherwise, you dont know where your failure came from.

This is not to saythat you shouldnt test against a real network or real database. This is indeed the domain of acceptance tests. But unit tests where you are testing a layer or just a small unit should be isolated and not concern itself outside the unit.

Typically i tend to "test" against the real deal, and then write a fake which simulates it for formal tests. You can see the procedure below for a mail client.

(But first, you can set up fixtures in a `conftest.py`)

In [134]:
%%file conftest.py
import pytest
import smtplib

@pytest.fixture(scope="module")
def smtp():
    return smtplib.SMTP("smtp.gmail.com", 587)

Writing conftest.py


This is what really happens:

In [136]:
import smtplib
xxx=smtplib.SMTP("smtp.gmail.com", 587)
xxx.ehlo()

(250,
 b'smtp.gmail.com at your service, [50.177.146.107]\nSIZE 35882577\n8BITMIME\nSTARTTLS\nENHANCEDSTATUSCODES\nPIPELINING\nCHUNKING\nSMTPUTF8')

In [137]:
xxx.noop()

(250, b'2.0.0 OK g6sm5963317qgd.5 - gsmtp')

In [138]:
%%file test_smtp.py

def test_ehlo(smtp):
    response, msg = smtp.ehlo()
    assert response == 250
    assert b"smtp.gmail.com" in msg

def test_noop(smtp):
    response, msg = smtp.noop()
    assert response == 250

Writing test_smtp.py


In [139]:
!py.test test_smtp.py

============================= test session starts ==============================
platform darwin -- Python 3.5.1, pytest-2.8.1, py-1.4.30, pluggy-0.3.1
rootdir: /Users/rahul/Projects/private/cs207/lecswithlabs/week5, inifile: 
plugins: cov-2.2.1
collected 2 items 

test_smtp.py ..

=========================== 2 passed in 0.18 seconds ===========================


Ok, but we shouldnt be stuck if the smtp server is down....so we look at the output from the previous and "fake" it for the `ehlo` and `noop` ops.

Notice below that its critical to "mock" the original object by momkey-patching in the `ehlo` and `noop` methods. The `monkeypatch` object in `py.test` will handle setting the original object's methods to the new function

In [140]:
%%file conftest.py
import pytest
import smtplib

def ehlo(smtpi):
    return (250, b'smtp.gmail.com BLA BLA')
def noop(smtpi):
    return (250, b'BLA BLA gsmtp')

@pytest.fixture(autouse=True)
def smtp(monkeypatch):
    #smtp_instance = smtplib.SMTP()
    monkeypatch.setattr(smtplib.SMTP, "ehlo", ehlo)
    monkeypatch.setattr(smtplib.SMTP, "noop", noop)
    return smtplib.SMTP()

Overwriting conftest.py


In [141]:
!py.test test_smtp.py

============================= test session starts ==============================
platform darwin -- Python 3.5.1, pytest-2.8.1, py-1.4.30, pluggy-0.3.1
rootdir: /Users/rahul/Projects/private/cs207/lecswithlabs/week5, inifile: 
plugins: cov-2.2.1
collected 2 items 

test_smtp.py ..

=========================== 2 passed in 4.48 seconds ===========================


This can also be done by using `mock` and `unittest`:

In [142]:
%%file test_smtp2.py

import unittest
import unittest.mock as mock
import smtplib

def ehlo():
    return (250, b'smtp.gmail.com BLA BLA')
def noop():
    return (250, b'BLA BLA gsmtp')

class MyTest(unittest.TestCase):
    
    def setUp(self):
        self.patcher = mock.patch("smtplib.SMTP")
        self.smtp = self.patcher.start()
        self.smtp.ehlo=ehlo
        self.smtp.noop=noop
        
    def tearDown(self):
        self.patcher.stop()

    def test_ehlo(self):
        response, msg = self.smtp.ehlo()
        assert response == 250
        assert b"smtp.gmail.com" in msg

    def test_noop(self):
        response, msg = self.smtp.noop()
        assert response == 250

Writing test_smtp2.py


In [143]:
!py.test test_smtp2.py

============================= test session starts ==============================
platform darwin -- Python 3.5.1, pytest-2.8.1, py-1.4.30, pluggy-0.3.1
rootdir: /Users/rahul/Projects/private/cs207/lecswithlabs/week5, inifile: 
plugins: cov-2.2.1
collected 2 items 

test_smtp2.py ..

=========================== 2 passed in 3.16 seconds ===========================


In your project, you will want to mock out your database connections in your unit tests. There are other libraries you can use as well, such as `minimock`.